# FFIG - A foreign function interface generator for `C++`

I want to write `C++` code and call it from Python without doing any extra work.

---

### For Project Managers
A language like Python can be written to resemble executable pseudo-code and is a great langauge for defining high-level acceptance criteria. 

Developer engagement is easy when requirements are code.

---

### For Clients 
It's easy to put together interactive demos with Python. Jupyter notebook allows one to display graphs, images, videos and tabulated data. 

The ability to change demos on the fly in response to client queries is seriously impressive.

---

### For Developers
I can break the edit-compile-link-test cycle with an interpreted scripting language. 

Once the design is right I can drive it into `C++`.

---

### Calling `C++` from Python

* Write some `C++` code.
* Define a `C-API` for binary compatibility.
* Solving problems with ownership.
* Define classes and functions that use a Foreign Function Interface to communicate with `C`.

---

### Code-generation

* Parse `C++` code.
* Transform parsed source code into a easy-to use form.
* Define a template that transforms parsed source code into `C`
* Define a template that transforms parsed source code into classes, functions and FFI calls.

---

### Extras

* Supporting Python 3.
* Supporting C++.

---

In [2]:
%%file Tree.hpp
#ifndef DEMOS_TREE_H
#define DEMOS_TREE_H

#include <memory>

class Tree {
  std::unique_ptr<Tree> left_;
  std::unique_ptr<Tree> right_;
  int data_ = 0;    
 
 public:
  Tree(int children) {
    if(children <=0) return; 
    left_ = std::make_unique<Tree>(children-1);
    right_ = std::make_unique<Tree>(children-1);
  } 
    
  Tree* left() { return left_.get(); }
  
  Tree* right() { return right_.get(); }
    
  int data() const { return data_; }
  
  void set_data(int x) { data_ = x; }
};

#endif // DEMOS_TREE_H

Overwriting Tree.hpp


Let's compile this to ensure we've not made any mistakes.

In [3]:
%%sh
clang++ -std=c++14 -fsyntax-only Tree.hpp

---

## Defining a C-API

We want a `C-API` so that we have a well-defined and portable binary interface. 

We'll have to re-model our code as C does not support classes.

Free functions with an extra leading argument for `this` should suffice.

In [4]:
%%file Tree_c.h

#define C_API extern "C" __attribute__((visibility("default")))

struct CTree_t;
typedef CTree_t* CTree;

C_API CTree Tree_create(int children);

C_API void Tree_dispose(CTree t);

C_API CTree Tree_left(CTree t);

C_API CTree Tree_right(CTree t);

C_API int Tree_data(CTree t);

C_API void Tree_set_data(CTree t, int x);

Overwriting Tree_c.h


In [5]:
%%file Tree_c.cpp

#include "Tree_c.h"
#include "Tree.hpp"

CTree Tree_create(int children) {
  auto tree = std::make_unique<Tree>(children);
  return reinterpret_cast<CTree>(tree.release());
}

void Tree_dispose(CTree t) {
  auto* tree = reinterpret_cast<Tree*>(t);
  delete tree;
}

CTree Tree_left(CTree t) {
  auto* tree = reinterpret_cast<Tree*>(t);
  return reinterpret_cast<CTree>(tree->left());
}

CTree Tree_right(CTree t) {
  auto* tree = reinterpret_cast<Tree*>(t);
  return reinterpret_cast<CTree>(tree->right());
}

int Tree_data(CTree t) {
  auto* tree = reinterpret_cast<Tree*>(t);
  return tree->data();
}

void Tree_set_data(CTree t, int x) {
  auto* tree = reinterpret_cast<Tree*>(t);
  tree->set_data(x);
}

Overwriting Tree_c.cpp


We can build this to create a shared library with our `C-API` symbols exposed.

In [6]:
%%file CMakeLists.txt

cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)
add_compile_options(-fvisibility=hidden)
add_library(Tree_c SHARED Tree_c.cpp)

Overwriting CMakeLists.txt


In [7]:
%%sh
rm -rf CMakeFiles/
rm CMakeCache.txt

cmake . -GNinja
cmake --build .
strip -x libTree_c.dylib

-- The C compiler identification is AppleClang 8.1.0.8020042
-- The CXX compiler identification is AppleClang 8.1.0.8020042
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been 

In [8]:
%%sh
nm -U libTree_c.dylib

0000000000001c30 T _Tree_create
0000000000001fc0 T _Tree_data
0000000000001ea0 T _Tree_dispose
0000000000001f00 T _Tree_left
0000000000001f60 T _Tree_right
0000000000002010 T _Tree_set_data


## Python interop

We can use Python's `ctypes` module to interact with the `C` shared-library.

In [9]:
import ctypes
ctypes.c_object_p = ctypes.POINTER(ctypes.c_void_p)

tree_lib = ctypes.cdll.LoadLibrary("libTree_c.dylib")

We need to tell ctypes about the arguments and return types of the functions.

By default ctypes assumes functions take no arguments and return an integer.

In [10]:
tree_lib.Tree_create.argtypes = [ctypes.c_int]
tree_lib.Tree_create.restype = ctypes.c_object_p

tree_lib.Tree_dispose.argtypes = [ctypes.c_object_p]
tree_lib.Tree_dispose.restype = None

tree_lib.Tree_left.argtypes = [ctypes.c_object_p]
tree_lib.Tree_left.restype = ctypes.c_object_p

tree_lib.Tree_right.argtypes = [ctypes.c_object_p]
tree_lib.Tree_right.restype = ctypes.c_object_p

tree_lib.Tree_data.argtypes = [ctypes.c_object_p]
tree_lib.Tree_data.restype = ctypes.c_int

tree_lib.Tree_set_data.argtypes = [ctypes.c_object_p, ctypes.c_int]
tree_lib.Tree_set_data.restype = None

We'll leave the string-related methods for now, interop there is not so easy.

Let's see what we can do with the fledgling Python API.

In [11]:
root = tree_lib.Tree_create(2)

In [12]:
tree_lib.Tree_data(root)

0

In [13]:
tree_lib.Tree_set_data(root, 42)
tree_lib.Tree_data(root)

42

In [14]:
tree_lib.Tree_dispose(root)

So far, so not-very-Pythonic.

We want classes!

In [15]:
class Tree(object):
    
    def __init__(self, children=None, _p=None):
        if _p:
            self._ptr = _p
            self.owner = False
        else:
            self._ptr = tree_lib.Tree_create(children)
            self.owner = True
        
    def __del__(self):
        if self.owner:
            tree_lib.Tree_dispose(self._ptr)
     
    def __repr__(self):
        return "<Tree data:{}>".format(self.data)
    
    @property
    def left(self):
        p = tree_lib.Tree_left(self._ptr)
        if not p: 
            return None
        return Tree(_p=p)
    
    @property
    def right(self):
        p = tree_lib.Tree_right(self._ptr)
        if not p: 
            return None
        
        return Tree(_p=p)
    
    @property
    def data(self):
        return tree_lib.Tree_data(self._ptr)
    
    @data.setter
    def data(self, x):
        tree_lib.Tree_set_data(self._ptr, x)

In [16]:
t = Tree(2)
t

<Tree data:0>

In [17]:
t.data = 42
t

<Tree data:42>

In [18]:
t.left

<Tree data:0>

In [20]:
t.left.data = 6

In [21]:
t.left

<Tree data:6>

This looks good but we our crude attempts at memory management will fail if we start working with temporaries.

In [ ]:
# This kills the kernel
# left = Tree(3).left.left
# left.data

Our Python classes don't know enough about the underlying C++ to do the memory management. 

Our C-API implementation needs a re-think.

## Defining a C-API (Again)

## A Safer Python API

---

# FFIG

### Parsing `C++` with libclang

### Jinja2 templates

### Generating a Python API with FFIG